In [2]:
"""
Saint Lux Store - Mini Tienda / Sistema de Ventas (versión actualizada)
Lenguajes Visuales II - Python + Tkinter + SQLite
"""
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog, filedialog
import sqlite3
from datetime import datetime
import csv
import os

DB_FILE = "saintlux_store.db"

# Colores - paleta Saint Lux
BG = "#000000"         # fondo negro
FG = "#FFFFFF"         # texto blanco
ACCENT = "#D4AF37"     # dorado (acento)
CARD = "#0f0f0f"       # panel oscuro

# Valores fijos solicitados
CATEGORY_OPTIONS = ["Remeras", "Camperas", "Pantalones", "Accesorios", "Calzados"]
SIZE_OPTIONS = ["XS", "S", "M", "L", "XL", "XXL"]

# -------------------- DB Helpers --------------------
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    # Productos con talla y categoría
    c.execute("""
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        code TEXT UNIQUE,
        name TEXT NOT NULL,
        category TEXT,
        size TEXT,
        price REAL NOT NULL,
        stock INTEGER NOT NULL DEFAULT 0,
        created_at TEXT
    )
    """)
    # Ventas
    c.execute("""
    CREATE TABLE IF NOT EXISTS sales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        total REAL NOT NULL,
        created_at TEXT
    )
    """)
    # Items por venta
    c.execute("""
    CREATE TABLE IF NOT EXISTS sale_items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sale_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        unit_price REAL,
        FOREIGN KEY (sale_id) REFERENCES sales(id),
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    """)
    conn.commit()
    conn.close()

# Product CRUD
def db_insert_product(code, name, category, size, price, stock):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("""
    INSERT INTO products (code, name, category, size, price, stock, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (code, name, category, size, price, stock, datetime.now().isoformat()))
    conn.commit()
    conn.close()

def db_update_product(pid, code, name, category, size, price, stock):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("""
    UPDATE products SET code=?, name=?, category=?, size=?, price=?, stock=? WHERE id=?
    """, (code, name, category, size, price, stock, pid))
    conn.commit()
    conn.close()

def db_delete_product(pid):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("DELETE FROM products WHERE id=?", (pid,))
    conn.commit()
    conn.close()

def db_fetch_products(filter_text=""):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    base = "SELECT id, code, name, category, size, price, stock, created_at FROM products"
    params = []
    if filter_text:
        base += " WHERE code LIKE ? OR name LIKE ? OR category LIKE ? OR size LIKE ?"
        params = [f"%{filter_text}%"] * 4
    base += " ORDER BY name"
    c.execute(base, params)
    rows = c.fetchall()
    conn.close()
    return rows

def db_get_product(pid):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT id, code, name, category, size, price, stock FROM products WHERE id=?", (pid,))
    row = c.fetchone()
    conn.close()
    return row

# Sales
def db_create_sale(total, items):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("INSERT INTO sales (total, created_at) VALUES (?, ?)", (total, datetime.now().isoformat()))
    sale_id = c.lastrowid
    for it in items:
        c.execute("INSERT INTO sale_items (sale_id, product_id, quantity, unit_price) VALUES (?, ?, ?, ?)",
                  (sale_id, it["product_id"], it["quantity"], it["unit_price"]))
        # decrease stock
        c.execute("UPDATE products SET stock = stock - ? WHERE id=?", (it["quantity"], it["product_id"]))
    conn.commit()
    conn.close()
    return sale_id

def db_fetch_sales(filter_from=None, filter_to=None):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    base = "SELECT id, total, created_at FROM sales"
    params = []
    clauses = []
    if filter_from:
        clauses.append("date(created_at) >= date(?)")
        params.append(filter_from)
    if filter_to:
        clauses.append("date(created_at) <= date(?)")
        params.append(filter_to)
    if clauses:
        base += " WHERE " + " AND ".join(clauses)
    base += " ORDER BY created_at DESC"
    c.execute(base, params)
    rows = c.fetchall()
    conn.close()
    return rows

def db_fetch_sale_items(sale_id):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("""
    SELECT si.id, si.product_id, p.name, si.quantity, si.unit_price
    FROM sale_items si
    LEFT JOIN products p ON p.id = si.product_id
    WHERE si.sale_id=?
    """, (sale_id,))
    rows = c.fetchall()
    conn.close()
    return rows

# -------------------- UI App --------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Saint Lux Store - Administración")
        self.geometry("1024x680")
        self.minsize(900, 600)
        # fondo principal
        self.configure(bg=BG)
        self.style = ttk.Style(self)
        self.setup_style()

        # logo top
        header = tk.Frame(self, bg=BG)
        header.pack(fill="x", pady=(8,2))
        # intentamos cargar logo
        self.logo_img = None
        logo_path = "saint_lux_logo.png"
        if os.path.exists(logo_path):
            try:
                # usar PhotoImage (acepta PNG). Si falla, se ignora y mostramos texto.
                self.logo_img = tk.PhotoImage(file=logo_path)
                lbl = tk.Label(header, image=self.logo_img, bg=BG)
                lbl.pack(side="left", padx=12)
            except Exception:
                lbl = tk.Label(header, text="Saint Lux Store", bg=BG, fg=FG, font=("Helvetica", 20, "bold"))
                lbl.pack(side="left", padx=12)
        else:
            lbl = tk.Label(header, text="Saint Lux Store", bg=BG, fg=FG, font=("Helvetica", 20, "bold"))
            lbl.pack(side="left", padx=12)

        # title
        title_lbl = tk.Label(header, text="Panel de administración - Saint Lux", bg=BG, fg=FG, font=("Helvetica", 12))
        title_lbl.pack(side="left", padx=8)

        # Menu principal (simple)
        menubar = tk.Menu(self, bg=BG, fg=FG)
        self.config(menu=menubar)
        filemenu = tk.Menu(menubar, tearoff=0, bg=BG, fg=FG)
        filemenu.add_command(label="Productos", command=lambda: self.show_frame("ProductsFrame"))
        filemenu.add_command(label="Punto de Venta", command=lambda: self.show_frame("SaleFrame"))
        filemenu.add_command(label="Historial de Ventas", command=lambda: self.show_frame("HistoryFrame"))
        filemenu.add_separator()
        filemenu.add_command(label="Exportar productos CSV", command=self.export_products_csv)
        filemenu.add_separator()
        filemenu.add_command(label="Salir", command=self.quit)
        menubar.add_cascade(label="Archivo", menu=filemenu)
        helpmenu = tk.Menu(menubar, tearoff=0, bg=BG, fg=FG)
        helpmenu.add_command(label="Acerca de", command=lambda: messagebox.showinfo("Acerca de", "Saint Lux Store - Lenguajes Visuales II"))
        menubar.add_cascade(label="Ayuda", menu=helpmenu)

        # contenedor de frames
        container = ttk.Frame(self, style="Card.TFrame")
        container.pack(fill="both", expand=True, padx=12, pady=8)

        self.frames = {}
        for F in (ProductsFrame, SaleFrame, HistoryFrame):
            frame = F(container, self)
            self.frames[F.__name__] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("ProductsFrame")

    def setup_style(self):
        try:
            self.style.theme_use("clam")
        except Exception:
            pass
        # Frame estilo tarjeta oscuro
        self.style.configure("Card.TFrame", background=CARD)
        self.style.configure("TLabel", background=CARD, foreground=FG)
        self.style.configure("Header.TLabel", background=CARD, foreground=FG, font=("Helvetica", 14, "bold"))
        # Botón accent (no siempre afecta todos los sistemas)
        self.style.configure("Accent.TButton", background=ACCENT, foreground=BG)
        # notas: ttk no siempre respeta background en todos los OS; se busca coherencia visual

    def show_frame(self, name):
        f = self.frames[name]
        if hasattr(f, "on_show"):
            f.on_show()
        f.tkraise()

    def export_products_csv(self):
        rows = db_fetch_products()
        if not rows:
            messagebox.showinfo("Exportar CSV", "No hay productos para exportar.")
            return
        path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files","*.csv")], title="Guardar productos como CSV")
        if not path:
            return
        try:
            with open(path, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["id","code","name","category","size","price","stock","created_at"])
                for r in rows:
                    writer.writerow(r)
            messagebox.showinfo("Exportar CSV", f"Exportado correctamente a:\n{path}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo exportar CSV:\n{e}")

# -------------------- Frame: Productos --------------------
class ProductsFrame(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, padding=12, style="Card.TFrame")
        self.controller = controller

        header = ttk.Label(self, text="Gestión de Productos (Saint Lux)", style="Header.TLabel")
        header.pack(anchor="w", pady=(0,8))

        top = ttk.Frame(self, style="Card.TFrame")
        top.pack(fill="x", pady=6)
        ttk.Label(top, text="Buscar:").pack(side="left")
        self.filter_var = tk.StringVar()
        search_entry = ttk.Entry(top, textvariable=self.filter_var, width=40)
        search_entry.pack(side="left", padx=6)
        search_entry.bind("<Return>", lambda e: self.load_products())
        ttk.Button(top, text="Buscar", command=self.load_products).pack(side="left", padx=4)
        ttk.Button(top, text="Nuevo producto", command=self.open_new_product).pack(side="right")

        cols = ("id","code","name","category","size","price","stock")
        self.tree = ttk.Treeview(self, columns=cols, show="headings", selectmode="browse", height=14)
        headings = ["ID","Código","Nombre","Categoría","Talla","Precio","Stock"]
        for c, h in zip(cols, headings):
            self.tree.heading(c, text=h)
        self.tree.column("id", width=50, anchor="center")
        self.tree.column("code", width=100)
        self.tree.column("name", width=300)
        self.tree.column("category", width=120)
        self.tree.column("size", width=80, anchor="center")
        self.tree.column("price", width=100, anchor="e")
        self.tree.column("stock", width=80, anchor="center")
        self.tree.pack(fill="both", expand=True, pady=8)

        btns = ttk.Frame(self, style="Card.TFrame")
        btns.pack(fill="x")
        ttk.Button(btns, text="Editar", command=self.edit_selected).pack(side="left")
        ttk.Button(btns, text="Eliminar", command=self.delete_selected).pack(side="left", padx=6)
        ttk.Button(btns, text="Actualizar catálogo", command=self.load_products).pack(side="left", padx=6)

        self.load_products()

    def on_show(self):
        self.load_products()

    def load_products(self):
        for r in self.tree.get_children():
            self.tree.delete(r)
        for row in db_fetch_products(self.filter_var.get().strip()):
            pid, code, name, category, size, price, stock, created = row
            self.tree.insert("", "end", values=(pid, code or "-", name, category or "-", size or "-", f"{price:.2f}", stock))

    def open_new_product(self):
        ProductDialog(self, title="Nuevo Producto - Saint Lux")

    def get_selected_id(self):
        sel = self.tree.selection()
        if not sel:
            return None
        return self.tree.item(sel[0])["values"][0]

    def edit_selected(self):
        pid = self.get_selected_id()
        if not pid:
            messagebox.showwarning("Atención", "Seleccione un producto para editar.")
            return
        ProductDialog(self, pid=pid, title="Editar Producto - Saint Lux")

    def delete_selected(self):
        pid = self.get_selected_id()
        if not pid:
            messagebox.showwarning("Atención", "Seleccione un producto para eliminar.")
            return
        if messagebox.askyesno("Confirmar", "¿Eliminar el producto seleccionado?"):
            try:
                db_delete_product(pid)
                messagebox.showinfo("Eliminado", "Producto eliminado correctamente.")
                self.load_products()
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo eliminar: {e}")

# Dialog for product create/edit
class ProductDialog(tk.Toplevel):
    def __init__(self, parent, pid=None, title="Producto"):
        super().__init__(parent)
        self.parent = parent
        self.pid = pid
        self.title(title)
        self.geometry("560x360")
        self.resizable(False, False)
        self.configure(bg=CARD)
        self.grab_set()

        frm = ttk.Frame(self, padding=12, style="Card.TFrame")
        frm.pack(fill="both", expand=True)

        ttk.Label(frm, text="Código (opcional)").grid(row=0, column=0, sticky="w", pady=6)
        self.code_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.code_var, width=40).grid(row=0, column=1, pady=6)

        ttk.Label(frm, text="Nombre *").grid(row=1, column=0, sticky="w", pady=6)
        self.name_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.name_var, width=40).grid(row=1, column=1, pady=6)

        ttk.Label(frm, text="Categoría").grid(row=2, column=0, sticky="w", pady=6)
        self.cat_var = tk.StringVar()
        ttk.Combobox(frm, textvariable=self.cat_var, values=CATEGORY_OPTIONS, state="readonly", width=38).grid(row=2, column=1, pady=6)

        ttk.Label(frm, text="Talla").grid(row=3, column=0, sticky="w", pady=6)
        self.size_var = tk.StringVar()
        ttk.Combobox(frm, textvariable=self.size_var, values=SIZE_OPTIONS, state="readonly", width=18).grid(row=3, column=1, sticky="w", pady=6)

        ttk.Label(frm, text="Precio *").grid(row=4, column=0, sticky="w", pady=6)
        self.price_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.price_var, width=20).grid(row=4, column=1, sticky="w", pady=6)

        ttk.Label(frm, text="Stock *").grid(row=5, column=0, sticky="w", pady=6)
        self.stock_var = tk.StringVar(value="0")
        ttk.Entry(frm, textvariable=self.stock_var, width=20).grid(row=5, column=1, sticky="w", pady=6)

        btns = ttk.Frame(frm)
        btns.grid(row=6, column=0, columnspan=2, pady=12)
        ttk.Button(btns, text="Guardar", command=self.save).pack(side="left")
        ttk.Button(btns, text="Cancelar", command=self.destroy).pack(side="left", padx=8)

        if self.pid:
            self.load_product()

    def load_product(self):
        row = db_get_product(self.pid)
        if not row:
            messagebox.showerror("Error", "Producto no encontrado.")
            self.destroy()
            return
        pid, code, name, category, size, price, stock = row
        self.code_var.set(code or "")
        self.name_var.set(name)
        self.cat_var.set(category or "")
        self.size_var.set(size or "")
        self.price_var.set(str(price))
        self.stock_var.set(str(stock))

    def validate(self):
        name = self.name_var.get().strip()
        if not name:
            messagebox.showwarning("Validación", "El nombre es obligatorio.")
            return False
        try:
            price = float(self.price_var.get())
            if price < 0:
                raise ValueError()
        except:
            messagebox.showwarning("Validación", "Precio inválido.")
            return False
        try:
            stock = int(self.stock_var.get())
            if stock < 0:
                raise ValueError()
        except:
            messagebox.showwarning("Validación", "Stock inválido (entero).")
            return False
        return True

    def save(self):
        if not self.validate():
            return
        code = self.code_var.get().strip() or None
        name = self.name_var.get().strip()
        cat = self.cat_var.get().strip() or None
        size = self.size_var.get().strip() or None
        price = float(self.price_var.get())
        stock = int(self.stock_var.get())
        try:
            if self.pid:
                db_update_product(self.pid, code, name, cat, size, price, stock)
                messagebox.showinfo("Guardado", "Producto actualizado.")
            else:
                db_insert_product(code, name, cat, size, price, stock)
                messagebox.showinfo("Guardado", "Producto creado.")
            self.parent.load_products()
            self.destroy()
        except sqlite3.IntegrityError:
            messagebox.showerror("Error", "El código de producto ya existe. Ingresá otro código.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo guardar: {e}")

# -------------------- Frame: Punto de Venta / Carrito --------------------
class SaleFrame(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, padding=12, style="Card.TFrame")
        self.controller = controller
        header = ttk.Label(self, text="Punto de Venta - Saint Lux", style="Header.TLabel")
        header.pack(anchor="w", pady=(0,8))

        top = ttk.Frame(self, style="Card.TFrame")
        top.pack(fill="x", pady=6)
        ttk.Label(top, text="Buscar producto:").pack(side="left")
        self.search_var = tk.StringVar()
        search_entry = ttk.Entry(top, textvariable=self.search_var, width=40)
        search_entry.pack(side="left", padx=6)
        search_entry.bind("<Return>", lambda e: self.load_products())
        ttk.Button(top, text="Buscar", command=self.load_products).pack(side="left", padx=4)
        ttk.Button(top, text="Limpiar búsqueda", command=self.clear_search).pack(side="left", padx=4)

        mid = ttk.Panedwindow(self, orient="horizontal")
        mid.pack(fill="both", expand=True, pady=8)

        # Productos disponibles (izquierda)
        left = ttk.Frame(mid, style="Card.TFrame", width=560)
        mid.add(left)
        cols = ("id","code","name","category","size","price","stock")
        self.prod_tree = ttk.Treeview(left, columns=cols, show="headings")
        headings = ["ID","Código","Nombre","Categoría","Talla","Precio","Stock"]
        for c,h in zip(cols, headings):
            self.prod_tree.heading(c, text=h)
        self.prod_tree.column("id", width=50, anchor="center")
        self.prod_tree.column("code", width=100)
        self.prod_tree.column("name", width=260)
        self.prod_tree.column("category", width=110)
        self.prod_tree.column("size", width=70, anchor="center")
        self.prod_tree.column("price", width=90, anchor="e")
        self.prod_tree.column("stock", width=70, anchor="center")
        self.prod_tree.pack(fill="both", expand=True)
        ttk.Button(left, text="Agregar al carrito", command=self.add_to_cart).pack(pady=6)

        # Carrito (derecha)
        right = ttk.Frame(mid, style="Card.TFrame", width=420)
        mid.add(right)
        ttk.Label(right, text="Carrito").pack(anchor="w")
        cart_cols = ("product_id","name","size","quantity","unit_price","subtotal")
        self.cart_tree = ttk.Treeview(right, columns=cart_cols, show="headings")
        headings = ["ID","Nombre","Talla","Cant","Precio","Subtotal"]
        for c,h in zip(cart_cols, headings):
            self.cart_tree.heading(c, text=h)
        self.cart_tree.column("product_id", width=40, anchor="center")
        self.cart_tree.column("name", width=160)
        self.cart_tree.column("size", width=60, anchor="center")
        self.cart_tree.column("quantity", width=60, anchor="center")
        self.cart_tree.column("unit_price", width=80, anchor="e")
        self.cart_tree.column("subtotal", width=80, anchor="e")
        self.cart_tree.pack(fill="both", expand=True)
        btns = ttk.Frame(right, style="Card.TFrame")
        btns.pack(fill="x", pady=6)
        ttk.Button(btns, text="Quitar seleccionado", command=self.remove_cart_item).pack(side="left")
        ttk.Button(btns, text="Vaciar carrito", command=self.clear_cart).pack(side="left", padx=6)
        ttk.Button(btns, text="Confirmar venta", command=self.confirm_sale).pack(side="right")

        # Totales
        bottom = ttk.Frame(self, style="Card.TFrame")
        bottom.pack(fill="x", pady=6)
        self.total_var = tk.StringVar(value="0.00")
        ttk.Label(bottom, text="Total:").pack(side="left")
        ttk.Label(bottom, textvariable=self.total_var, font=("Helvetica", 12, "bold")).pack(side="left", padx=6)

        # Data structures
        self.cart = {}  # product_id -> {product_id, name, size, quantity, unit_price}

        self.load_products()

    def on_show(self):
        self.load_products()
        self.clear_cart()

    def load_products(self):
        for r in self.prod_tree.get_children():
            self.prod_tree.delete(r)
        for row in db_fetch_products(self.search_var.get().strip()):
            pid, code, name, category, size, price, stock, created = row
            self.prod_tree.insert("", "end", values=(pid, code or "-", name, category or "-", size or "-", f"{price:.2f}", stock))

    def clear_search(self):
        self.search_var.set("")
        self.load_products()

    def add_to_cart(self):
        sel = self.prod_tree.selection()
        if not sel:
            messagebox.showwarning("Atención", "Seleccione un producto para agregar.")
            return
        vals = self.prod_tree.item(sel[0])["values"]
        pid = vals[0]
        name = vals[2]
        size = vals[4]
        unit_price = float(vals[5])
        stock = int(vals[6])

        q = simpledialog.askinteger("Cantidad", f"Ingrese cantidad (stock disponible: {stock})", initialvalue=1, minvalue=1, maxvalue=stock)
        if not q:
            return

        if pid in self.cart:
            if self.cart[pid]["quantity"] + q > stock:
                messagebox.showwarning("Stock", "No hay stock suficiente para esa cantidad.")
                return
            self.cart[pid]["quantity"] += q
        else:
            self.cart[pid] = {"product_id": pid, "name": name, "size": size, "quantity": q, "unit_price": unit_price}

        self.refresh_cart_view()

    def refresh_cart_view(self):
        for r in self.cart_tree.get_children():
            self.cart_tree.delete(r)
        total = 0.0
        for pid, it in self.cart.items():
            subtotal = it["quantity"] * it["unit_price"]
            total += subtotal
            self.cart_tree.insert("", "end", values=(it["product_id"], it["name"], it["size"], it["quantity"], f"{it['unit_price']:.2f}", f"{subtotal:.2f}"))
        self.total_var.set(f"{total:.2f}")

    def remove_cart_item(self):
        sel = self.cart_tree.selection()
        if not sel:
            messagebox.showwarning("Atención", "Seleccione un item del carrito para quitar.")
            return
        pid = self.cart_tree.item(sel[0])["values"][0]
        if pid in self.cart:
            del self.cart[pid]
            self.refresh_cart_view()

    def clear_cart(self):
        self.cart.clear()
        self.refresh_cart_view()

    def confirm_sale(self):
        if not self.cart:
            messagebox.showwarning("Atención", "El carrito está vacío.")
            return
        total = sum(it["quantity"] * it["unit_price"] for it in self.cart.values())
        if not messagebox.askyesno("Confirmar venta", f"Total: {total:.2f}\n¿Confirmar y registrar la venta?"):
            return
        items = []
        for it in self.cart.values():
            items.append({"product_id": it["product_id"], "quantity": it["quantity"], "unit_price": it["unit_price"]})
        try:
            sale_id = db_create_sale(total, items)
            messagebox.showinfo("Venta registrada", f"Venta #{sale_id} registrada. Total: {total:.2f}")
            self.clear_cart()
            self.load_products()  # refresh stock display
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo registrar la venta:\n{e}")

# -------------------- Frame: Historial de ventas --------------------
class HistoryFrame(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, padding=12, style="Card.TFrame")
        self.controller = controller
        header = ttk.Label(self, text="Historial de Ventas - Saint Lux", style="Header.TLabel")
        header.pack(anchor="w", pady=(0,8))

        filterf = ttk.Frame(self, style="Card.TFrame")
        filterf.pack(fill="x")
        ttk.Label(filterf, text="Desde (YYYY-MM-DD)").pack(side="left")
        self.from_var = tk.StringVar()
        ttk.Entry(filterf, textvariable=self.from_var, width=12).pack(side="left", padx=6)
        ttk.Label(filterf, text="Hasta (YYYY-MM-DD)").pack(side="left")
        self.to_var = tk.StringVar()
        ttk.Entry(filterf, textvariable=self.to_var, width=12).pack(side="left", padx=6)
        ttk.Button(filterf, text="Filtrar", command=self.load_sales).pack(side="left", padx=6)
        ttk.Button(filterf, text="Exportar ventas CSV", command=self.export_sales_csv).pack(side="right")

        cols = ("id","total","created_at")
        self.tree = ttk.Treeview(self, columns=cols, show="headings", selectmode="browse", height=14)
        for c in cols:
            self.tree.heading(c, text=c.capitalize())
        self.tree.column("id", width=80, anchor="center")
        self.tree.column("total", width=120, anchor="e")
        self.tree.column("created_at", width=260)
        self.tree.pack(fill="both", expand=True, pady=8)

        bot = ttk.Frame(self, style="Card.TFrame")
        bot.pack(fill="x")
        ttk.Button(bot, text="Ver detalle", command=self.view_detail).pack(side="left")
        ttk.Button(bot, text="Actualizar catálogo", command=self.load_sales).pack(side="left", padx=6)

        self.load_sales()

    def on_show(self):
        self.load_sales()

    def load_sales(self):
        for r in self.tree.get_children():
            self.tree.delete(r)
        rows = db_fetch_sales(filter_from=self.from_var.get().strip() or None, filter_to=self.to_var.get().strip() or None)
        for r in rows:
            sid, total, created = r
            self.tree.insert("", "end", values=(sid, f"{total:.2f}", created))

    def view_detail(self):
        sel = self.tree.selection()
        if not sel:
            messagebox.showwarning("Atención", "Seleccione una venta para ver detalle.")
            return
        sid = self.tree.item(sel[0])["values"][0]
        items = db_fetch_sale_items(sid)
        if not items:
            messagebox.showinfo("Detalle", "No se encontraron items para esa venta.")
            return
        text = f"Detalle Venta #{sid}\n\n"
        sum_ = 0.0
        for it in items:
            iid, pid, name, qty, unit = it
            subtotal = qty * unit
            sum_ += subtotal
            text += f"- {name} | Cant: {qty} | Unit: {unit:.2f} | Subtotal: {subtotal:.2f}\n"
        text += f"\nTotal: {sum_:.2f}"
        messagebox.showinfo(f"Venta #{sid}", text)

    def export_sales_csv(self):
        rows = db_fetch_sales(filter_from=self.from_var.get().strip() or None, filter_to=self.to_var.get().strip() or None)
        if not rows:
            messagebox.showinfo("Exportar CSV", "No hay ventas para exportar con ese filtro.")
            return
        path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files","*.csv")], title="Guardar ventas como CSV")
        if not path:
            return
        try:
            with open(path, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["sale_id","total","created_at","item_product_id","item_name","item_quantity","item_unit_price"])
                for sale in rows:
                    sid, total, created = sale
                    items = db_fetch_sale_items(sid)
                    if items:
                        for it in items:
                            iid, pid, name, qty, unit = it
                            writer.writerow([sid, f"{total:.2f}", created, pid, name, qty, f"{unit:.2f}"])
                    else:
                        writer.writerow([sid, f"{total:.2f}", created, "", "", "", ""])
            messagebox.showinfo("Exportado", f"Exportado correctamente a:\n{path}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo exportar:\n{e}")

# -------------------- Main --------------------
if __name__ == "__main__":
    init_db()
    app = App()
    app.mainloop()
